<a href="https://colab.research.google.com/github/gauravJ443/Control-Flow-Assignment/blob/main/Dispenser_ARM_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
import datetime
import time
import os
import matplotlib.pyplot as plt
import seaborn as sns

# --- Authentication ---
auth_url = "https://noc.atgl.adani.digital/user-management/api/v1/apikey/token"
auth_headers = {'api-key' : "9d6f6672c18e98ca339bba63ef17f2ca617430ad"}

response = requests.post(auth_url, headers=auth_headers)
auth_response_json = response.json()

# --- Get Asset IDs if authentication is successful ---
asset_ids = []
if response.status_code == 200:
    access_token = auth_response_json['data']['accessToken']
    asset_base_url = "https://noc.atgl.adani.digital/amm/api/assetModel/5140/assets?page={}&paged=false&size=3000"
    asset_headers = {"IdToken": access_token}

    seen_ids = set()
    page = 1
    while True:
        asset_url = asset_base_url.format(page)
        asset_response = requests.get(asset_url, headers=asset_headers)
        asset_data = asset_response.json()

        if not asset_data['content']:
            break

        for asset in asset_data['content']:
            if asset['id'] not in seen_ids:
                asset_ids.append(asset['id'])
                seen_ids.add(asset['id']) # Add to seen_ids here

        page += 1

    # print(f"Found {len(asset_ids)} asset IDs.")

# --- Fetch Raw Data for each Asset ID if authentication is successful ---
if response.status_code == 200:
    access_token = auth_response_json['data']['accessToken']
    raw_data_url = "https://noc.atgl.adani.digital/dataexplorer/v1/raw/data"
    raw_data_headers = {"Content-Type": "application/json; charset=utf-8","idToken": access_token}

    step = 150
    no_of_days = 2
    df_combined = None

    for day_offset in reversed(range(no_of_days)):
        start_time_day = time.time()
        needed_date = datetime.date.today() - datetime.timedelta(days=day_offset)
        from_date = needed_date.strftime('%Y-%m-%d 00:00:00')
        to_date = needed_date.strftime('%Y-%m-%d 23:59:59')
        print(f"Fetching data for: {needed_date}")

        df_day = None # Initialize df_day for each day

        for i in range(0, len(asset_ids), step):
            start_time_asset = time.time()
            selected_assets_batch = asset_ids[i:i+step]

            payload = {
                "start_date": from_date,
                "end_date": to_date,
                "timezone" : "Asia/Kolkata",
                "gapfill": True,
                "granularity": 5,
                "ui_mode": True,
                "aggregation": "AVG",
                "asset_categories": [{
                    "ARM_A": {
                        "asset": selected_assets_batch,
                        "tags": ["DIS_MFM_TOT", "DIS_MB_TOT"]
                    }
                }]
            }

            raw_data_response = requests.post(raw_data_url, headers=raw_data_headers, json=payload)
            kpi_data = raw_data_response.json()

            if 'data' in kpi_data and len(kpi_data['data']) > 0:
                df_batch = pd.DataFrame(kpi_data['data'])
                if df_day is None:
                    df_day = df_batch
                else:
                    df_day = pd.merge(df_day, df_batch, how='outer', on='timestamp')
                # print(f"Data received for batch starting at index {i}. Size: {df_batch.size}")
            else:
                print(f"No data received for asset batch starting at index {i}")
                # print("----------Payload-----------")
                # print(payload)
                # print("----------API Response-----------")
                # print(kpi_data)

            end_time_asset = time.time()
            # print(f'Time taken to get asset data for batch: {end_time_asset - start_time_asset:.2f} sec')

        # Concatenate data for the current day with the combined data
        if df_day is not None:
            if df_combined is None:
                df_combined = df_day
            else:
                df_combined = pd.concat([df_combined, df_day], axis=0)
        else:
             print(f"No data collected for {needed_date}")

        print(f'Time taken to get data for day {needed_date}: {time.time() - start_time_day:.2f} sec')

    # Optional: Save the combined dataframe to a CSV file
    # if df_combined is not None:
    #     df_combined.to_csv("ARM_A_DIS_SALES_combined.csv", index=False)
    #     print("Combined data saved to ARM_A_DIS_SALES_combined.csv")
    # else:
    #     print("No data was collected to save.")

Fetching data for: 2025-08-16
Time taken to get data for day 2025-08-16: 21.71 sec
Fetching data for: 2025-08-17
Time taken to get data for day 2025-08-17: 21.47 sec
Fetching data for: 2025-08-18
No data received for asset batch starting at index 0
No data received for asset batch starting at index 150
No data received for asset batch starting at index 300
No data received for asset batch starting at index 450
No data received for asset batch starting at index 600
No data received for asset batch starting at index 750
No data received for asset batch starting at index 900
No data collected for 2025-08-18
Time taken to get data for day 2025-08-18: 4.96 sec


In [1]:
import requests
import pandas as pd
import numpy as np
import datetime
import time
import os
import matplotlib.pyplot as plt
import seaborn as sns

# --- Authentication ---
auth_url = "https://noc.atgl.adani.digital/user-management/api/v1/apikey/token"
auth_headers = {'api-key' : "9d6f6672c18e98ca339bba63ef17f2ca617430ad"}

response = requests.post(auth_url, headers=auth_headers)
auth_response_json = response.json()

# --- Get Asset IDs if authentication is successful ---
asset_ids_5140 = []
asset_ids_5255 = []
if response.status_code == 200:
    access_token = auth_response_json['data']['accessToken']
    asset_base_url = "https://noc.atgl.adani.digital/amm/api/assetModel/{}/assets?page={}&paged=false&size=3000"
    asset_headers = {"IdToken": access_token}

    # Fetch asset IDs for model 5140
    seen_ids_5140 = set()
    page = 1
    while True:
        asset_url = asset_base_url.format(5140, page)
        asset_response = requests.get(asset_url, headers=asset_headers)
        asset_data = asset_response.json()

        if not asset_data['content']:
            break

        for asset in asset_data['content']:
            if asset['id'] not in seen_ids_5140:
                asset_ids_5140.append(asset['id'])
                seen_ids_5140.add(asset['id'])

        page += 1

    # Fetch asset IDs for model 5255
    seen_ids_5255 = set()
    page = 1
    while True:
        asset_url = asset_base_url.format(5255, page)
        asset_response = requests.get(asset_url, headers=asset_headers)
        asset_data = asset_response.json()

        if not asset_data['content']:
            break

        for asset in asset_data['content']:
            if asset['id'] not in seen_ids_5255:
                asset_ids_5255.append(asset['id'])
                seen_ids_5255.add(asset['id'])

        page += 1

    print(f"Found {len(asset_ids_5140)} asset IDs for model 5140 and {len(asset_ids_5255)} asset IDs for model 5255.")

# --- Fetch Raw Data for each Asset ID if authentication is successful ---
if response.status_code == 200:
    access_token = auth_response_json['data']['accessToken']
    raw_data_url = "https://noc.atgl.adani.digital/dataexplorer/v1/raw/data"
    raw_data_headers = {"Content-Type": "application/json; charset=utf-8","idToken": access_token}

    step = 150
    no_of_days = 2
    df_combined = None

    # Combine asset IDs for iteration, keeping track of their original model
    combined_asset_ids = []
    for asset_id in asset_ids_5140:
        combined_asset_ids.append({'id': asset_id, 'model': 5140})
    for asset_id in asset_ids_5255:
        combined_asset_ids.append({'id': asset_id, 'model': 5255})


    for day_offset in reversed(range(no_of_days)):
        start_time_day = time.time()
        needed_date = datetime.date.today() - datetime.timedelta(days=day_offset)
        from_date = needed_date.strftime('%Y-%m-%d 00:00:00')
        to_date = needed_date.strftime('%Y-%m-%d 23:59:59')
        print(f"Fetching data for: {needed_date}")

        df_day = None # Initialize df_day for each day

        for i in range(0, len(combined_asset_ids), step):
            start_time_asset = time.time()
            selected_assets_batch = combined_asset_ids[i:i+step]

            # Separate asset IDs by model for the payload
            asset_ids_batch_5140 = [asset['id'] for asset in selected_assets_batch if asset['model'] == 5140]
            asset_ids_batch_5255 = [asset['id'] for asset in selected_assets_batch if asset['model'] == 5255]

            asset_categories_payload = {}
            if asset_ids_batch_5140:
                asset_categories_payload["ARM_A"] = {
                    "asset": asset_ids_batch_5140,
                    "tags": ["DIS_MFM_TOT", "DIS_MB_TOT"]
                }
            if asset_ids_batch_5255:
                 asset_categories_payload["ARM_B"] = {
                    "asset": asset_ids_batch_5255,
                    "tags": ["DIS_MFM_TOT", "DIS_MB_TOT"]
                }


            payload = {
                "start_date": from_date,
                "end_date": to_date,
                "timezone" : "Asia/Kolkata",
                "gapfill": True,
                "granularity": 5,
                "ui_mode": True,
                "aggregation": "AVG",
                "asset_categories": [asset_categories_payload] # Wrap in a list
            }

            raw_data_response = requests.post(raw_data_url, headers=raw_data_headers, json=payload)
            kpi_data = raw_data_response.json()

            if 'data' in kpi_data and len(kpi_data['data']) > 0:
                df_batch = pd.DataFrame(kpi_data['data'])
                if df_day is None:
                    df_day = df_batch
                else:
                    df_day = pd.merge(df_day, df_batch, how='outer', on='timestamp')
                # print(f"Data received for batch starting at index {i}. Size: {df_batch.size}")
            else:
                print(f"No data received for asset batch starting at index {i}")
                # print("----------Payload-----------")
                # print(payload)
                # print("----------API Response-----------")
                # print(kpi_data)


            end_time_asset = time.time()
            # print(f'Time taken to get asset data for batch: {end_time_asset - start_time_asset:.2f} sec')

        # Concatenate data for the current day with the combined data
        if df_day is not None:
            if df_combined is None:
                df_combined = df_day
            else:
                df_combined = pd.concat([df_combined, df_day], axis=0)
        else:
             print(f"No data collected for {needed_date}")


        print(f'Time taken to get data for day {needed_date}: {time.time() - start_time_day:.2f} sec')

    # Optional: Save the combined dataframe to a CSV file
    # if df_combined is not None:
    #     df_combined.to_csv("ARM_A_DIS_SALES_combined.csv", index=False)
    #     print("Combined data saved to ARM_A_DIS_SALES_combined.csv")
    # else:
    #     print("No data was collected to save.")

Found 1014 asset IDs for model 5140 and 998 asset IDs for model 5255.
Fetching data for: 2025-08-18
Time taken to get data for day 2025-08-18: 49.45 sec
Fetching data for: 2025-08-19
No data received for asset batch starting at index 0
No data received for asset batch starting at index 150
No data received for asset batch starting at index 300
No data received for asset batch starting at index 450
No data received for asset batch starting at index 600
No data received for asset batch starting at index 750
No data received for asset batch starting at index 900
No data received for asset batch starting at index 1050
No data received for asset batch starting at index 1200
No data received for asset batch starting at index 1350
No data received for asset batch starting at index 1500
No data received for asset batch starting at index 1650
No data received for asset batch starting at index 1800
No data received for asset batch starting at index 1950
No data collected for 2025-08-19
Time take

In [2]:
df_combined

,timestamp,cha183_dis_pkr68a.dis_mfm_tot,cha183_dis_pkr68a.dis_mb_tot,cha183_dis_pkr57a.dis_mfm_tot,cha183_dis_pkr57a.dis_mb_tot,cha183_dis_pkr67a.dis_mfm_tot,cha183_dis_pkr67a.dis_mb_tot,cha183_dis_pkr56a.dis_mfm_tot,cha183_dis_pkr56a.dis_mb_tot,apv300_dis_kdpkr08a.dis_mfm_tot,...,vsp636_dis_cdpkr14b.dis_mfm_tot,vsp636_dis_cdpkr14b.dis_mb_tot,vmp886_dis_udpkr37b.dis_mfm_tot,vmp886_dis_udpkr37b.dis_mb_tot,wdj155_dis_pkr228b.dis_mfm_tot,wdj155_dis_pkr228b.dis_mb_tot,vgb398_dis_pkr89b.dis_mfm_tot,vgb398_dis_pkr89b.dis_mb_tot,vgb398_dis_pkr94b.dis_mfm_tot,vgb398_dis_pkr94b.dis_mb_tot
0,2025-08-18 00:00:00,None,None,1603515.75,527353.25,1139318.25,3811550.00,3674327.75,719304.19,2346775.50,...,83925.23,83920.33,362663.03,362600.59,240597.25,240596.00,1790820.00,282679.69,631701.88,1608184.88
1,2025-08-18 00:05:00,None,None,1603515.75,527353.25,1139318.25,3811550.00,3674327.75,719304.19,2346775.50,...,83925.23,83920.33,362663.03,362600.59,240648.17,240596.00,1790820.00,282679.69,631701.88,1608184.88
2,2025-08-18 00:10:00,None,None,1603515.75,527353.25,1139318.25,3811550.00,3674327.75,719304.19,2346775.50,...,83925.23,83920.33,362663.03,362600.59,240659.91,240658.64,1790820.00,282679.69,631701.88,1608184.88
3,2025-08-18 00:15:00,4184921.50,None,1603515.75,527353.25,1139318.25,3811550.00,3674327.75,719304.19,2346775.50,...,83925.23,83920.33,362663.03,362600.59,240696.73,240658.64,1790820.00,282679.69,631701.88,1608184.88
4,2025-08-18 00:20:00,4184921.50,None,1603515.75,527353.25,1139318.25,3811550.00,3674327.75,719304.19,2346775.50,...,83925.23,83920.33,362663.03,362600.59,240699.66,240698.41,1790820.00,282679.69,631701.88,1608184.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,2025-08-18 23:35:00,4185648.50,185676.39,1604936.38,528767.44,1140433.50,3812665.00,3674327.75,719304.19,2348203.00,...,84165.95,84161.02,363448.03,363385.59,242930.89,242929.63,1791087.50,282947.16,632244.25,1608727.13
284,2025-08-18 23:40:00,4185648.50,185676.39,1604936.38,528767.44,1140433.50,3812665.00,3674327.75,719304.19,2348203.00,...,84165.95,84161.02,363448.03,363385.59,242975.00,242929.63,1791087.50,282947.16,632244.25,1608727.13
285,2025-08-18 23:45:00,4185648.50,185676.39,1604936.38,528767.44,1140433.50,3812665.00,3674327.75,719304.19,2348203.00,...,84165.95,84161.02,363448.03,363385.59,242995.53,242994.27,1791087.50,282947.16,632244.25,1608727.13
286,2025-08-18 23:50:00,4185648.50,185676.39,1604936.38,528767.44,1140433.50,3812665.00,3674327.75,719304.19,2348203.00,...,84165.95,84161.02,363448.03,363385.59,243058.78,243057.53,1791087.50,282947.16,632244.25,1608727.13
